In [ ]:
model_hist_data =
model_fut_data =
obs_data =
this_month =
mode = 'addqq'

Variable =

In [ ]:
percentiles = xr.DataArray(np.linspace(0, 1, num=101))

In [ ]:
# Processing historical model data ...

ds_mon_hist_month = model_hist_data.sel(time=model_hist_data.time.dt.month.isin([this_month])) # select month
ds_mon_hist_month_mean = ds_mon_hist_month.mean('time')
ds_mon_hist_month_mean_interp = ds_mon_hist_month_mean[Variable]

ds_mon_hist_month1 = ds_mon_hist_month
var = ds_mon_hist_month1[Variable]

ds_mon_hist_month_perc = var.quantile(
    percentiles,
    dim="time",
    interpolation='linear',
    keep_attrs=True,
    skipna=True
).compute() # percentile calculation

ds_mon_hist_month_perc_interp = ds_mon_hist_month_perc

In [1]:
# Processing future model data

ds_mon_fut_month = model_fut_data.sel(time=model_fut_data.time.dt.month.isin([this_month]))
ds_mon_fut_month_mean = ds_mon_fut_month.mean('time')
ds_mon_fut_month_mean_interp = ds_mon_fut_month_mean[Variable]

ds_mon_fut_month1=ds_mon_fut_month.chunk(chunks={"lat": 10, "lon":10,"time":-1})
var = ds_mon_fut_month1[Variable]

ds_mon_fut_month_perc = var.quantile(
    percentiles,
    dim="time",
    interpolation='linear',
    keep_attrs=True,
    skipna=True
).compute()

ds_mon_fut_month_perc_interp = ds_mon_fut_month_perc

IndentationError: unexpected indent (291637631.py, line 15)

In [ ]:
# Calculating change between future and historical scenario ...
change_month_add = ds_mon_fut_month_perc_interp - ds_mon_hist_month_perc_interp # change for additive bias correction
change_month_mul = ds_mon_fut_month_perc_interp.where(ds_mon_fut_month_perc_interp>10**-20) / ds_mon_hist_month_perc_interp.where(ds_mon_hist_month_perc_interp>10**-20) # change for multiplicative bias correction
change_month_mul=change_month_mul.fillna(10**-20) 

In [ ]:
# Calculating change between future and historical scenario ...
change_month_add = ds_mon_fut_month_perc_interp - ds_mon_hist_month_perc_interp # change for additive bias correction
change_month_mul = ds_mon_fut_month_perc_interp.where(ds_mon_fut_month_perc_interp>10**-20) / ds_mon_hist_month_perc_interp.where(ds_mon_hist_month_perc_interp>10**-20) # change for multiplicative bias correction
change_month_mul = change_month_mul.fillna(10**-20) 

In [ ]:
# Processing observational data ...
ds_mon_obs_month = obs_data.sel(time=obs_data.time.dt.month.isin([this_month])) # find reverse percentile for obs data
ds_mon_obs_month1 = ds_mon_obs_month
ds_mon_obs_month_perc = ds_mon_obs_month1[Variable].quantile(
    percentiles,dim="time",
    interpolation='linear',
    keep_attrs=True,
    skipna=True
).compute()
rank_obs_month=ds_mon_obs_month.load().rank(dim="time",pct=True,keep_attrs=True)*100

In [ ]:
# Doing bias correction
change_month_add_stack = change_month_add
change_month_mul_stack = change_month_mul
ds_mon_obs_month_perc_stack = ds_mon_obs_month_perc
obs_month_loc_stack = rank_obs_month[Variable]
ds_mon_obs_month_stack = ds_mon_obs_month[Variable]
temp_var = xr.DataArray(
    np.empty(np.shape(obs_month_loc_stack)),
    dims=["time",],
    coords=obs_month_loc_stack.coords
)
if (mode=='addqq'):
###### Applying the change factor 
    for tt in range(ds_mon_obs_month_stack.time.size):
        print('{0} of {1}'.format(tt,len(ds_mon_obs_month_stack.time)))
        temp_var1 = obs_month_loc_stack[tt,:].astype('int')
        temp_var2 = temp_var1.where(temp_var1 < 100, 99)
        temp_var3 = abs(temp_var2 - obs_month_loc_stack[tt,:])
        change_factor=(1-temp_var3)*change_month_add_stack[temp_var2,:]+temp_var3*change_month_add_stack[temp_var2+1,:]
        temp_var[tt,:] = ds_mon_obs_month_stack[tt,:] + change_factor

    obs_month_adjusted = temp_var
        
    ratio1 = obs_month_adjusted.mean('time').where(abs(obs_month_adjusted.mean('time'))>10**-20)-ds_mon_obs_month.mean('time').where(abs(ds_mon_obs_month.mean('time'))>10**-20)
    ratio2 = ds_mon_fut_month_mean_interp.where(abs(ds_mon_fut_month_mean_interp)>10**-20)-ds_mon_hist_month_mean_interp.where(abs(ds_mon_hist_month_mean_interp)>10**-20)

    bias_factor=ratio2-ratio1
    bias_factor=bias_factor.fillna(10**-20)

    MO_corrected = obs_month_adjusted + bias_factor[Variable]    
    MO_corrected = MO_corrected.astype('float32')

elif (mode=='mulqq'):
    ###### Applying the change factor 

        for tt in range(ds_mon_obs_month_stack.time.size):

            print('{0} of {1}'.format(tt,len(ds_mon_obs_month_stack.time)))

            temp_var1 = obs_month_loc_stack[tt,:].astype('int').drop('z')

            temp_var2=temp_var1.where(temp_var1<100,99)

            temp_var3=abs(temp_var2-obs_month_loc_stack[tt,:])

            change_factor=(1-temp_var3)*change_month_mul_stack[temp_var2,:]+temp_var3*change_month_mul_stack[temp_var2+1,:] 

            temp_var[tt,:]=ds_mon_obs_month_stack[tt,:]*change_factor

        obs_month_adjusted=temp_var.unstack()


###### bias correction of the data                
        ratio1=obs_month_adjusted.mean('time').where(obs_month_adjusted.mean('time')>10**-20)/ds_mon_obs_month.mean('time').where(ds_mon_obs_month.mean('time')>10**-20)

        ratio2=ds_mon_fut_month_mean_interp.where(ds_mon_fut_month_mean_interp>10**-20)/ds_mon_hist_month_mean_interp.where(ds_mon_hist_month_mean_interp>10**-20)
  
        bias_factor=ratio2/ratio1

        bias_factor=bias_factor.fillna(10**-20)

        MO_corrected = obs_month_adjusted*bias_factor